In [41]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

page = 1
BASE_URL = "https://www.siasat.com/"
country = "bangladesh"
initial_url = f'{BASE_URL}?s=&q={country}#gsc.tab=0&gsc.q={country}&gsc.page={page}'

all_news_links = []


#TAKING LINKS FROM FIRST 4 PAGES

for page in range(1,4):

    url = f'https://www.siasat.com/?s=&q=bangladesh#gsc.tab=0&gsc.q=bangladesh&gsc.page={page}'
    # chrome_options = Options()
    # chrome_options.add_argument("--headless")
    # driver = webdriver.Chrome(options=chrome_options)
    driver = webdriver.Chrome()
    driver.get(url)

    MAX_RETRIES = 3
    consecutive_failures = 0

#THE WEBSITE DOESN'T LOAD PROPERLY IN THE FIRST TRY. BUT, LOADS IN ALL SECOND TRIES SO FAR.

    while consecutive_failures < MAX_RETRIES:

        try:

            main_div = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/div/div/div/div[1]/div/div/div[1]/div[6]/div[2]/div[1]/div')

            if main_div:

#                     print(main_div)
                    news = main_div.find_elements(By.CSS_SELECTOR, 'a')

                    for n in news:

                            news_text = n.get_attribute('data-ctorig')

                            if news_text in all_news_links:
                                continue
                            else:
                                all_news_links.append(news_text)

            break 


        except NoSuchElementException:

            consecutive_failures +=1
            time.sleep(10)


    driver.quit()


#For some reason, a None object gets appended to the list. So, removing that here.
modified_news_links = [i for i in all_news_links if i is not None]

#SCRAPING THE LINKS

data_list = []
counter = 0

for url in modified_news_links:
        
        if 'bangladesh' in url or 'bangladeshi' in url or 'bangladeshs' in url:
        
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            #TITLE

            title_tag = soup.find('meta', {'property':'og:title'})
            title = title_tag.get('content') if title_tag else 'Title not found'
            title_translation = 'None'

            #DATE

            date_data= soup.find('meta',{'property':'article:modified_time'}).get('content')

            if date_data:

                only_date = date_data.split('T')[0]
                only_time = date_data.split('T')[1]
                time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
                cleaned_date = f"{only_date},{time}"

                source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M") + timedelta(hours = 5, minutes = 30) 
                bangladesh_localtime = source_localtime + timedelta(minutes=30)

            else:

                date_data = 'Date data not found'


            #AUTHOR

            author_tags = soup.find('span', class_ = 'author-meta single-author with-avatars')
            author = author_tags.text.strip().split('Follow')[0].strip() if author_tags else 'Author not found'


            #CONTENT SUMMARY

            content_summary_tag = soup.find('h2', class_ = 'entry-sub-title')

            content_summary = content_summary_tag.text if content_summary_tag else 'Content Summary not found'

            summary_translation = 'None'

            #CONTENT

            content = []

            main_content_id = soup.find('article',{'id':'the-post'})

            if main_content_id:

                all_paras = main_content_id.find_all('p')

                for para in all_paras:
                    content.append(para.text)

                full_content = ''.join(content)
                full_content = full_content.split('Get the news updates')[0]

            else:
                full_content = 'Content not found'

            content_translation = 'None'


            data_dict = {
            "url": url,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content Not Found" and title != 'Title not found'):

                if data_dict not in data_list:
                        # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')


df = pd.DataFrame(data_list)
df.head()


csv_filename = f"{country}_Siasat_Daily.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["bangladesh_localtime"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Link 1 added
Link 2 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added
Link 18 added
Link 19 added
Link 20 added
Link 21 added
Link 22 added
Link 23 added
Link 25 added
Link 26 added
Link 27 added
Link 28 added
Link 29 added


In [42]:
df.head()

,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.siasat.com/bangladesh-embraces-ind...,Bangladesh embraces Indian medico's 'Save Girl...,Pune: Thousands of people and medicos in Bangl...,"Dr. Rakh, who was invited for a 10-day long gu...",None,None,None,Indo-Asian News Service,bangladesh,2024-02-25 12:34:00,2024-02-25 13:04:00
1,https://www.siasat.com/bangladesh-pacer-mustaf...,Bangladesh pacer Mustafizur Rahman hospitalise...,Dhaka: Experienced left-arm pacer Mustafizur R...,Accident occurred while they were participatin...,None,None,None,Press Trust of India,bangladesh,2024-02-18 15:27:00,2024-02-18 15:57:00
2,https://www.siasat.com/bangladeshi-nobel-laure...,Bangladeshi Nobel laureate Yunus charge-sheete...,Dhaka: Bangladesh’s anti-graft agency on Thurs...,Court has set March 3 for an indictment hearin...,None,None,None,Press Trust of India,bangladesh,2024-02-01 21:07:00,2024-02-01 21:37:00
3,https://www.siasat.com/assam-would-have-been-p...,Assam would have been part of Bangladesh if in...,Guwahati: Union Home Minister Amit Shah on Sat...,"""Earlier, history was written and taught to us...",None,None,None,Press Trust of India,bangladesh,2024-01-20 21:32:00,2024-01-20 22:02:00
4,https://www.siasat.com/will-protect-myanmar-bo...,Will protect India-Myanmar border with fencing...,Guwahati: Home Minister Amit Shah on Saturday ...,"""The India-Myanmar border will be protected li...",None,None,None,Press Trust of India,bangladesh,2024-01-20 19:22:00,2024-01-20 19:52:00
